##  Hello all. Welcome to XAI day by Open Data science conference, New Delhi.

**Give us an upvote if you find it useful.**

Warm regards,
Team ODSC New Delhi

#This Notebook contains a collection of methods to weave explainability into AI systems.
It can come handy while studying models,debugging ML pipelines and for exploratory purposes. 
A system is only as good as its creator. Today, we acknowledge the limitations of our craft and attempt to understand it better.

In [ ]:
#Importing required packages.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
%matplotlib inline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import matplotlib


## Dataset

Source: https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009

This dataset is also available from the UCI machine learning repository, https://archive.ics.uci.edu/ml/datasets/wine+quality , I just shared it to kaggle for convenience. (If I am mistaken and the public license type disallowed me from doing so, I will take this down if requested.)

Content
For more information, read [Cortez et al., 2009].
Input variables (based on physicochemical tests):
1 - fixed acidity
2 - volatile acidity
3 - citric acid
4 - residual sugar
5 - chlorides
6 - free sulfur dioxide
7 - total sulfur dioxide
8 - density
9 - pH
10 - sulphates
11 - alcohol
Output variable (based on sensory data):
12 - quality (score between 0 and 10)

Columns description
* fixed acidity:most acids involved with wine or fixed or nonvolatile (do not evaporate readily)
* 
* volatile acidity:the amount of acetic acid in wine, which at too high of levels can lead to an unpleasant, vinegar taste
* 
* citric acid:found in small quantities, citric acid can add 'freshness' and flavor to wines
* 
* residual sugar:the amount of sugar remaining after fermentation stops, it's rare to find wines with less than 1 gram/liter and wines with greater than 45 grams/liter are considered sweet
* 
* chlorides:the amount of salt in the wine
* 
* free sulfur dioxide:the free form of SO2 exists in equilibrium between molecular SO2 (as a dissolved gas) and bisulfite ion; it prevents microbial growth and the oxidation of wine
* 
* total sulfur dioxide:amount of free and bound forms of S02; in low concentrations, SO2 is mostly undetectable in wine, but at free SO2 concentrations over 50 ppm, SO2 becomes evident in the nose and taste of wine
* 
* density:the density of water is close to that of water depending on the percent alcohol and sugar content
* 
* pH:describes how acidic or basic a wine is on a scale from 0 (very acidic) to 14 (very basic); most wines are between 3-4 on the pH scale
* 
* sulphates:a wine additive which can contribute to sulfur dioxide gas (S02) levels, wich acts as an antimicrobial and antioxidant
* 
* alcohol:the percent alcohol content of the wine
* 
* quality:output variable (based on sensory data, score between 0 and 10)

In [ ]:
#Loading dataset
wine = pd.read_csv('../input/winequality-red.csv')

In [ ]:
#Let's check how the data is distributed
wine.head()

In [ ]:
#Information about the data columns
wine.info()

#### **Before moving to sophisticated methods,Let's start with the basics.Graphical representations are very powerful for explaining otherwise complex things.Humans have evolved to process visual stimuli faster than numbers,so lets look at the distribution of some of our variables**

# Visual EDA

In [ ]:
#Here we see that fixed acidity does not give any specification to classify the quality.
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'fixed acidity', data = wine)

In [ ]:
#Here we see that its quite a downing trend in the volatile acidity as we go higher the quality 
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'volatile acidity', data = wine)

In [ ]:
#Composition of citric acid go higher as we go higher in the quality of the wine
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'citric acid', data = wine)

In [ ]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'residual sugar', data = wine)

In [ ]:
#Composition of chloride also go down as we go higher in the quality of the wine
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'chlorides', data = wine)

In [ ]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'free sulfur dioxide', data = wine)

In [ ]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'total sulfur dioxide', data = wine)

In [ ]:
#Sulphates level goes higher with the quality of wine
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'sulphates', data = wine)

In [ ]:
#Alcohol level also goes higher as te quality of wine increases
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'alcohol', data = wine)

## Simple Logistic Regression

In [ ]:
wine.quality.describe()

In [ ]:
#Making binary classificaion for the response variable.
#Dividing wine as good and bad by giving the limit for the quality
bins = (2,6,8)
group_names = ['bad', 'good']
wine['quality'] = pd.cut(wine['quality'], bins = bins, labels = group_names)

In [ ]:
#Now lets assign a labels to our quality variable
label_quality = LabelEncoder() 

In [ ]:
#Bad becomes 0 and good becomes 1 
wine['quality'] = label_quality.fit_transform(wine['quality'])

In [ ]:
wine['quality'].value_counts()

In [ ]:
sns.countplot(wine['quality'])

In [ ]:
#Now seperate the dataset as response variable and feature variabes
X = wine.drop('quality', axis = 1)
y = wine.quality

In [ ]:
#Train and Test splitting of data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
#Applying Standard scaling to get optimized result
sc = StandardScaler()

In [ ]:
# X_train = sc.fit_transform(X_train)
# X_test = sc.fit_transform(X_test)
#commented on purpose to demonstrate something

In [ ]:
type(X_train)

    ### Simple Logistic Regression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
pred_logreg = logreg.predict(X_test)

In [ ]:
#Testing time
print(classification_report(y_test,pred_logreg))

A decent accuracy!

In [ ]:
#Confusion matrix for the random forest classification
print(confusion_matrix(y_test, pred_logreg))

# Explain to me like I'm 5

In [ ]:
import eli5
eli5.show_weights(logreg)
#uses permutation importance to compute feature weights

That gives us the weights associated to each feature, that can be seen as the contribution of each feature into predicting that the class will be y=1 (the client will subscribe after the campaign).

The names for each features aren't really helping though, we can pass a list of column names to eli5 but we'll need to do a little gymnastics first to extract names from our preprocessor in the pipeline (since we've generated new features on the fly with the one hot encoder)

In [ ]:
feat_names=wine.columns[:-1].tolist()

In [ ]:
feat_names

In [ ]:
eli5.show_weights(logreg, feature_names=feat_names)

So this is for the whole model.Let's nitpick a particular observation

In [ ]:
import numpy as np
i = np.random.randint(1,100)

In [ ]:
i

In [ ]:
X_test.iloc[i]

In [ ]:
y_test.iloc[i]

In [ ]:
eli5.show_prediction(logreg, 
                     X_test.iloc[i],
                     feature_names=feat_names, show_feature_values=True)

ELI5 understands text processing utilities from scikit-learn and can highlight text data accordingly. It also allows to debug scikit-learn pipelines which contain HashingVectorizer, by undoing hashing.

XGBoost - show feature importances and explain predictions of XGBClassifier, XGBRegressor and xgboost.Booster.

LightGBM - show feature importances and explain predictions of LGBMClassifier and LGBMRegressor.

CatBoost - show feature importances of CatBoostClassifier and CatBoostRegressor.

lightning - explain weights and predictions of lightning classifiers and regressors.

sklearn-crfsuite. ELI5 allows to check weights of sklearn_crfsuite.CRF models.

Keras - explain predictions of image classifiers via Grad-CAM visualizations.

Source: https://eli5.readthedocs.io/en/latest/overview.html
    

## LIME :Local Interpretable Model-Agnostic Explanations
Lime works on the principle of local fidelity ie that a point behaves in the same manner as that of its immediate neighbors. Fort his purpose,Lime is lighter than ELI5 and often faster.

In [ ]:
rfc=RandomForestClassifier(n_estimators=200)
rfm = rfc.fit(X_train,y_train)
pred_rfc= rfc.predict(X_test)

In [ ]:
#Testing time
print(classification_report(y_test,pred_rfc))

In [ ]:
print(confusion_matrix(y_test, pred_rfc))

In [ ]:
#looking at eli5 first
eli5.show_weights(rfm, 
                  feature_names=feat_names)

In [ ]:
from lime.lime_tabular import LimeTabularExplainer


The parameters passed to the explainer are:

Training set sans one hot encoding
mode: the explainer can be used for classification or regression
feature_names: list of labels for our features
categorical_features: list of indexes of categorical features
categorical_names: dict mapping each index of categorical feature to a list of corresponding labels
dicretize_continuous: will discretize numerical values into buckets that can be used for explanation. For instance it can tell us that the decision was made because distance is in bucket [5km, 10km] instead of telling us distance is an importante feature.

In [ ]:
X_train.head()

In [ ]:
explainer = LimeTabularExplainer(X_train.values,
                                 mode="classification",
                                 feature_names=X_train.columns.tolist(),
                                 categorical_names=None,
                                 categorical_features=None,
                                 discretize_continuous=True,
                                 random_state=42)

In [ ]:
prob=lambda x:rfm.predict_proba(X_test[[i]]).astype(float)


The explainer is all set up to explain observations!
Let's use the old i

In [ ]:
X_test.iloc[i]

In [ ]:
#prediction function
pred_fn = lambda x: rfm.predict_proba(x).astype(float)

# Hi I'm LIME,I'm the infamous gossip monger. If I dont know you,I'll take your neighbour's word for it!! 
[Kriti is awesome though! Be like KD] <3

In [ ]:
explanation = explainer.explain_instance(X_test.iloc[i], pred_fn)
explanation.show_in_notebook(show_table=True, show_all=False,)
print(explanation.score)

This is simple.sign=direction of relationship and coefficients= weights

# Moving on,let's talk about the concept of fairness.How do you define fair?

![](http://www.publichealthnotes.com/wp-content/uploads/2017/05/Equality-Vs-Equity..final-edit-1.jpg)

In [ ]:
!pip install https://github.com/adebayoj/fairml/archive/master.zip
# Installing another package called fairML

> The basic idea behind FairML is to measure a model’s dependence on its inputs by changing them. If a small change to an input feature dramatically changes the output, the model is sensitive to the feature.
> Think sensitivty analysis that we studied in economics/Calculus in schools.
Remember orthogonal vectors?
Source: https://blog.fastforwardlabs.com/2017/03/09/fairml-auditing-black-box-predictive-models.html

In [ ]:
from fairml import audit_model
importances, _ = audit_model(rfm.predict, X_test)
print(importances)

In [ ]:
#inbuilt methods to visualize it
total, _ = audit_model(logreg.predict, X_test)
# print feature importance
print(total)

# generate feature dependence plot
from fairml import plot_dependencies
fig = plot_dependencies(
    total.median(),
    reverse_values=False,
    title="FairML feature dependence"
)
plt.savefig("fairml_ldp.eps", transparent=False, bbox_inches='tight')

**References:**
* Bonus resource: a Textbook on Fairness in ML: https://fairmlbook.org/pdf/fairmlbook.pdf
* Image source: https://www.publichealthnotes.com/equity-vs-equality/
* https://blog.fastforwardlabs.com/2017/03/09/fairml-auditing-black-box-predictive-models.html
